### Imports

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install path.py
!pip install pytorch3d
clear_output()

In [ ]:
import numpy as np
import math
import random
import os
import plotly.graph_objects as go
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils

from path import Path

random.seed = 42

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

path = Path("ModelNet10")

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]

clear_output()
classes = {folder: i for i, folder in enumerate(folders)}
classes

{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

In [ ]:
def default_transforms():
    return transforms.Compose([
                               PointSampler(1024),
                               Normalize(),
                               RandomNoise(),
                               ToSorted(),
                               ToTensor()
    ])

!gdown https://drive.google.com/uc?id=1CVwVxdfUfP6TRcVUjjJvQeRcgCGcnSO_
from helping import *
clear_output()

### Data Preprocessing

In [ ]:
with open(path/"dresser/train/dresser_0001.off", 'r') as f:
    verts, faces = read_off(f)

i, j, k = np.array(faces).T
x, y, z = np.array(verts).T

# len(x)

In [ ]:
# visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()
# visualize_rotate([go.Scatter3d(x=x, y=y, z=z, mode='markers')]).show()

In [ ]:
# pcshow(x, y, z)

In [ ]:
pointcloud = PointSampler(1024)((verts, faces))
# pcshow(*pointcloud.T)

norm_pointcloud = Normalize()(pointcloud)
# pcshow(*norm_pointcloud.T)

noisy_pointcloud = RandomNoise()(norm_pointcloud)
# pcshow(*noisy_pointcloud.T)

rot_pointcloud = RandomRotation_z()(noisy_pointcloud)
# pcshow(*rot_pointcloud.T)

sorted_pointcloud = ToSorted()(rot_pointcloud)
# pcshow(*sorted_pointcloud.T)

tensor_pointcloud = ToTensor()(sorted_pointcloud)

### Creating Loaders

In [ ]:
# # all classes

# BATCH_SIZE = 32

# def trs():
#     return transforms.Compose([
#                                 PointSampler(1024),
#                                 RandomNoise(),
#                                 RandomRotation_z(),
#                                 Normalize(),
#                                 ToSorted(),
#                                 ToTensor()
#     ])

# train_ds = PointCloudData(path, transform=trs())
# valid_ds = PointCloudData(path, valid=True, folder='test', transform=trs())

# train_loader = DataLoader(train_ds, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
# valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, drop_last=True)

# !mkdir dataloaders
# torch.save(train_loader, 'dataloaders/trainloader.pth')
# torch.save(valid_loader, 'dataloaders/validloader.pth')
# !cp -r dataloaders_32_sorted drive/MyDrive/Thesis/dataloaders

# # !cp -r drive/MyDrive/dataloaders ./

In [ ]:
# # all classes - no ToSorted

# BATCH_SIZE = 32

# def trs():
#     return transforms.Compose([
#                                 PointSampler(1024),
#                                 RandomNoise(),
#                                 RandomRotation_z(),
#                                 Normalize(),
#                                 ToTensor()
#     ])

# train_ds = PointCloudData(path, transform=trs())
# valid_ds = PointCloudData(path, valid=True, folder='test', transform=trs())

# train_loader = DataLoader(train_ds, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
# valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, drop_last=True)
# !mkdir dataloaders_no_sort
# torch.save(train_loader, 'dataloaders_no_sort/trainloader.pth')
# torch.save(valid_loader, 'dataloaders_no_sort/validloader.pth')
# !cp -r dataloaders_no_sort drive/MyDrive/Thesis/dataloaders

# # !cp -r drive/MyDrive/Thesis/dataloaders/dataloaders_no_sort ./

In [ ]:
# # BEDS

# BATCH_SIZE = 32

# def trs():
#     return transforms.Compose([
#                                PointSampler(1024),
#                                Normalize(),
#                                RandomRotation_z(),
#                                RandomNoise(),
#                                ToSorted(),
#                                ToTensor()
#     ])

# beds_train_dataset = PointCloudData(path, folders=['bed'], transform=trs())
# beds_valid_dataset = PointCloudData(path, folder='test', folders=['bed'], transform=trs())

# beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
# beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

# !mkdir dataloader_beds
# torch.save(beds_train_loader, 'dataloader_beds/trainloader.pth')
# torch.save(beds_valid_loader, 'dataloader_beds/validloader.pth')
# !cp -r dataloader_beds drive/MyDrive/Thesis/dataloaders

# # !cp -r drive/MyDrive/Thesis/dataloaders/dataloader_beds ./

In [ ]:
# BEDS "No Rotation"

BATCH_SIZE = 32

def trs():
    return transforms.Compose([
                               PointSampler(1024),
                               Normalize(),
                               RandomNoise(),
                               ToSorted(),
                               ToTensor()
    ])

beds_train_dataset = PointCloudData(path, folders=['bed'], transform=trs())
beds_valid_dataset = PointCloudData(path, folder='test', folders=['bed'], transform=trs())

beds_train_loader = DataLoader(dataset=beds_train_dataset, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, num_workers=4, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_no_rot_4
torch.save(beds_train_loader, 'dataloader_beds_no_rot_4/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_no_rot_4/validloader.pth')
!cp -r dataloader_beds_no_rot_4 drive/MyDrive/Thesis/dataloaders

# !cp -r drive/MyDrive/Thesis/dataloaders/dataloader_beds_no_noise_rot_4 ./

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [ ]:
# BEDS "No noise", "No Rotation"

BATCH_SIZE = 32

def trs():
    return transforms.Compose([
                               PointSampler(1024),
                               Normalize(),
                               ToSorted(),
                               ToTensor()
    ])

beds_train_dataset = PointCloudData(path, folders=['bed'], transform=trs())
beds_valid_dataset = PointCloudData(path, folder='test', folders=['bed'], transform=trs())

beds_train_loader = DataLoader(dataset=beds_train_dataset, num_workers=4, shuffle=True, batch_size=BATCH_SIZE)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, num_workers=4, batch_size=BATCH_SIZE)

!mkdir dataloader_beds_no_noise_rot_4
torch.save(beds_train_loader, 'dataloader_beds_no_noise_rot_4/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_no_noise_rot_4/validloader.pth')
!cp -r dataloader_beds_no_noise_rot_4 drive/MyDrive/Thesis/dataloaders

# !cp -r drive/MyDrive/Thesis/dataloaders/dataloader_beds_no_noise_rot_4 ./

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [ ]:
BATCH_SIZE = 32

def trs_small():
    return transforms.Compose([
                               PointSampler(256),
                               ToSorted(),
                               RandomNoise(),
                               Normalize(),
                               ToTensor()
    ])

beds_train_dataset_small = PointCloudData(path, folders=['bed'], transform=trs_small())
beds_valid_dataset_small = PointCloudData(path, folder='test', folders=['bed'], transform=trs_small())

beds_train_loader_small = DataLoader(dataset=beds_train_dataset_small, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader_small = DataLoader(dataset=beds_valid_dataset_small, num_workers=4, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_small_4
torch.save(beds_train_loader_small, 'dataloader_beds_small_4/trainloader.pth')
torch.save(beds_valid_loader_small, 'dataloader_beds_small_4/validloader.pth')
!cp -r dataloader_beds_small_4 drive/MyDrive/Thesis/dataloaders

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [ ]:
class PointCloudDataPre(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms(), folders=None):
        self.root_dir = root_dir
        if not folders:
            folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform
        self.valid = valid
        self.pcs = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    with open(new_dir/file, 'r') as f:
                        verts, faces = read_off(f)
                    sample['pc'] = self.transforms((verts, faces))
                    sample['category'] = category
                    self.pcs.append(sample)

    def __len__(self):
        return len(self.pcs)

    def __getitem__(self, idx):
        pointcloud = self.pcs[idx]['pc']
        category = self.pcs[idx]['category']
        return pointcloud, self.classes[category]

In [ ]:
# BATCH_SIZE = 32

# def trs_small():
#     return transforms.Compose([
#                                PointSampler(256),
#                                ToSorted(),
#                                RandomNoise(),
#                                Normalize(),
#                                ToTensor()
#     ])

# beds_train_dataset_small = PointCloudDataPre(path, folders=['bed'], transform=trs_small())
# beds_valid_dataset_small = PointCloudDataPre(path, folder='test', folders=['bed'], transform=trs_small())

# beds_train_loader_small_pre = DataLoader(dataset=beds_train_dataset_small, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
# beds_valid_loader_small_pre = DataLoader(dataset=beds_valid_dataset_small, num_workers=4, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_small_pre_4
torch.save(beds_train_loader_small_pre, 'dataloader_beds_small_pre_4/trainloader.pth')
torch.save(beds_valid_loader_small_pre, 'dataloader_beds_small_pre_4/validloader.pth')
!cp -r dataloader_beds_small_pre_4 drive/MyDrive/Thesis/dataloaders

In [ ]:
BATCH_SIZE = 32

def trs():
    return transforms.Compose([
                               PointSampler(1024),
                               ToSorted(),
                            #    RandomNoise(),
                               Normalize(),
                               ToTensor()
    ])

beds_train_dataset_pre = PointCloudDataPre(path, folders=['bed'], transform=trs())
beds_valid_dataset_pre = PointCloudDataPre(path, folder='test', folders=['bed'], transform=trs())

beds_train_loader_pre = DataLoader(dataset=beds_train_dataset_pre, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader_pre = DataLoader(dataset=beds_valid_dataset_pre, num_workers=4, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_pre_4
torch.save(beds_train_loader_pre, 'dataloader_beds_pre_4/trainloader.pth')
torch.save(beds_valid_loader_pre, 'dataloader_beds_pre_4/validloader.pth')
!cp -r dataloader_beds_pre_4 drive/MyDrive/Thesis/dataloaders

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [ ]:
BATCH_SIZE = 32

static_trs = transforms.Compose([
                               PointSampler(1024),
                               ToSorted(),
                               Normalize(),
])

later_trs = transforms.Compose([
                                RandomNoise(),
                                ToTensor()
])

beds_train_dataset_pre = PointCloudDataBoth(path, folders=['bed'], static_transform=static_trs, later_transform=later_trs)
beds_valid_dataset_pre = PointCloudDataBoth(path, folder='test', folders=['bed'], static_transform=static_trs)

beds_train = DataLoader(dataset=beds_train_dataset_pre, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid = DataLoader(dataset=beds_valid_dataset_pre, num_workers=4, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_both_4
torch.save(beds_train, 'dataloader_beds_both_4/trainloader.pth')
torch.save(beds_valid, 'dataloader_beds_both_4/validloader.pth')
!cp -r dataloader_beds_both_4 drive/MyDrive/Thesis/dataloaders

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [ ]:
BATCH_SIZE = 32

static_trs = transforms.Compose([
                               PointSampler(1024),
                               ToSorted(),
                               Normalize(),
])

later_trs = transforms.Compose([
                                RandomNoise(),
                                ToTensor()
])

beds_train_dataset_pre = PointCloudDataBoth(path, folders=['bed'], static_transform=static_trs, later_transform=later_trs)
beds_valid_dataset_pre = PointCloudDataBoth(path, folder='test', folders=['bed'], static_transform=static_trs)

beds_train = DataLoader(dataset=beds_train_dataset_pre, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid = DataLoader(dataset=beds_valid_dataset_pre, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_both
torch.save(beds_train, 'dataloader_beds_both/trainloader.pth')
torch.save(beds_valid, 'dataloader_beds_both/validloader.pth')
!cp -r dataloader_beds_both drive/MyDrive/Thesis/dataloaders